In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import networkx as nx

from statsmodels.tsa.vector_ar.var_model import VAR

import warnings
warnings.filterwarnings('ignore')

# VAR - Vector Autoregression

$$
\mathbf{y}_t=\sum_{p=1}^P\mathbf{B}_p\mathbf{y}_{t-p}+\mathbf{e}_t
$$

$$
\mathbf{Y}=\mathbf{X}\mathbf{B}+\mathbf{E}
$$

In [ ]:
# get some comodity data

tickers = ['BZ=F', 'CL=F', 'GC=F']

# Download data
df = yf.download(tickers, period='7d', interval='1m')['Adj Close']

# rename columns
df.columns = ['Brent', 'WTI', 'Gold']

# forward fill nan values
df = df.ffill()

# Create log returns
rt = np.log(df).diff().dropna()

df.plot(linewidth=0.7)


In [ ]:
order = 10

# Fit VAR model
model = VAR(df)
results = model.fit(order)
display(results.summary())

## Impulse Response Analysis

Impulse Response Analysis is used to understand how a variable in a multivariate system responds over time to shocks (or impulses) in each variable, including itself.

$$
\mathbf{Y}_t=\mu+\sum_{i=0}^\infty\mathbf{\Phi}_iu_{t-i}
$$

Note the plot function is flexible and can plot only variables of interest if so desired:

```python
irf.plot(impulse='BZ=F')
```

In [ ]:
irf = results.irf(20)
fig_1 = irf.plot(orth=False)
fig_2 =irf.plot_cum_effects(orth=False);

fig_1.tight_layout(pad=0.7)
fig_2.tight_layout(pad=0.7)

# set minutes as xlabel
fig_1.axes[2].set_xlabel('Minutes')
fig_2.axes[2].set_xlabel('Minutes')
fig_1.axes[3].set_xlabel('Minutes')
fig_2.axes[3].set_xlabel('Minutes')

## Forecast Error Variance Decomposition (FEVD)


Forecast Error Variance Decomposition (FEVD) provides an understanding of how much of the forecast error variance of each variable in the multivariate system can be attributed to shocks (or innovations) to each variable in the system. In simpler terms, it helps to decipher the extent to which future values of a variable can be explained by its own shocks versus shocks to other variables in the model.

$$
\omega_{jk,i}(H)=\frac{\sum_{h=0}^{H-1}(e_j'\Theta_ie_h)^2}{\sum_{h=0}^{H-1}e_j'\Phi_i\Sigma_u\Phi_i'e_h}
$$


**Interpretation:**

If a large proportion of the forecast error variance of a variable is attributed to its own shocks, the variable is said to have a strong internal dynamics.
Conversely, if a significant portion of the variance is due to shocks to other variables, it suggests that external factors (represented by these other variables) play a major role in influencing the variable.

**Usage:**

FEVD is particularly useful in policy analysis, economic forecasting, and financial market studies. It helps in understanding the dynamics of economic systems and the interdependencies among different economic variables.

In [ ]:
fevd = results.fevd(15)
fevd.summary()
fig = fevd.plot()
fig.tight_layout()


# Total spillovers

The total
spillover index measures the contribution of spillovers
of volatility shocks across four asset classes to the total
forecast error variance.


$$
S^g(H)=100\cdot\frac{1}{N}\sum_{i,j=1}^N\omega_{ij,g}(H)
$$